In [2]:
from langchain.llms import OpenAI

In [3]:
import os
os.environ["OPENAI_API_KEY"] =  os.getenv("OPENAI_API_KEY")



 if value is towards 1 more creative but risky
temperature value  towards 1---> more creative more risk wrong o/p

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
llm = OpenAI(temperature = 0.6)

In [7]:
text = "what is capital of India"
print(llm.predict(text))



The capital of India is New Delhi.


In [9]:
from langchain import HuggingFaceHub
#llm_HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})
#output = llm_HuggingFaceHub.predict("Can you tell me capital of India")
#print(output)

### Prompt Template and LLMChain

In [15]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
prompt_template = PromptTemplate(

  input_variables=['country'],
  template='tell me capital of this {country}'
)
prompt_template.format(country = 'India')

chain = LLMChain(llm=llm,prompt = prompt_template)
chain.run('India')
#llm.predict(prompt_template=prompt_template,text=['India'])

'\n\nThe capital of India is New Delhi.'

### Combining multiple chains using Simple Seq Chain

In [19]:
from langchain.chains import SimpleSequentialChain
captital_prompt = PromptTemplate(
  input_variables=['country'],
  template='Tell me the capital of the {country}'
)

captial_chain = LLMChain(llm=llm,prompt = captital_prompt)

###second prompt 
fms_prompt = PromptTemplate(
  input_variables=['capital'],
  template="Suggest me some amazing places to visit in {capital}"
)

fms_chain = LLMChain(llm=llm,prompt = fms_prompt)
chain = SimpleSequentialChain(chains = [captial_chain,fms_chain])
chain.run('USA')

' Some amazing places to visit in Washington, D.C. include:\n\n1. The National Mall - This iconic stretch of land is home to numerous monuments and memorials, including the Lincoln Memorial and Washington Monument.\n\n2. The Smithsonian Museums - There are 19 museums and galleries that make up the Smithsonian Institution, including the National Air and Space Museum, National Museum of Natural History, and the National Museum of African American History and Culture.\n\n3. The White House - Take a tour of the iconic home of the President of the United States and learn about its history and significance.\n\n4. Georgetown - This historic neighborhood is filled with charming cobblestone streets, unique shops, and delicious restaurants.\n\n5. Arlington National Cemetery - Pay your respects to fallen soldiers at this peaceful and solemn cemetery.\n\n6. The National Zoo - This free zoo is part of the Smithsonian and is home to over 2,000 animals, including giant pandas.\n\n7. The Kennedy Cente

## Seq chain 

In [24]:
from langchain.chains import SequentialChain

captital_prompt = PromptTemplate(
  input_variables=['country'],
  template='Tell me the capital of the {country}'
)

captial_chain = LLMChain(llm=llm,prompt = captital_prompt,output_key = 'capital')

###second prompt 
fms_prompt = PromptTemplate(
  input_variables=['capital'],
  template="Suggest me some amazing places to visit in {capital}"
)

fms_chain = LLMChain(llm=llm,prompt = fms_prompt,output_key = 'places')
chain = SequentialChain(chains = [captial_chain,fms_chain],
input_variables = ['country'],
output_variables =['capital','places'])
chain({'country':"India"})


{'country': 'India',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': " Here are some amazing places to visit there:\n\n1. Red Fort: This iconic monument is a must-visit for its stunning architecture and historical significance. Built in the 17th century, it served as the residence of Mughal emperors.\n\n2. Qutub Minar: Another famous landmark of Delhi, Qutub Minar is a 73-meter tall tower built in the 12th century. It is a UNESCO World Heritage Site and a popular tourist spot.\n\n3. India Gate: This war memorial is a symbol of national pride and is a popular spot for picnics and evening walks. It also has a beautiful garden surrounding it.\n\n4. Lotus Temple: This beautiful temple, shaped like a lotus flower, is a popular attraction for its unique architecture and serene atmosphere. It is open to people of all religions.\n\n5. Akshardham Temple: This grand temple complex is a popular religious spot and also houses a cultural center with exhibitions and shows.\n\n6. Hum

## ChatModel ChatOpenAI

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
chat_llm = ChatOpenAI(temperature=0.6, model="gpt-3.5-turbo")

In [27]:
## schema - systemmessage
chat_llm(
[
  SystemMessage(content="You are a comedian AI assistant"),
  HumanMessage(content="Please provide some comedy punch lines on AI")
])

AIMessage(content='1. "Why did the AI break up with its robot girlfriend? It couldn\'t handle her emotional baggage!"\n2. "I asked my AI assistant to tell me a joke, and it replied, \'I\'m sorry, Dave, I\'m afraid I can\'t do that.\'"\n3. "I told my AI to make me a sandwich, and it responded with, \'I can\'t do that, but I can help you order one online!\'"\n4. "I tried to have a deep conversation with my AI, but all it wanted to talk about was binary code. Talk about being one-dimensional!"\n5. "I asked my AI to predict the future, and it said, \'I see a lot of cat videos in your future.\' Looks like it knows me well!"', response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 25, 'total_tokens': 183}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-890695ad-0bd2-48a8-a026-5cb63b7124a4-0')

# Prompt Template + LLM + Output Parsers

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [30]:
class commaseperateop(BaseOutputParser):
  def parse(self,text:str):
    return text.strip().split(",")

In [33]:
template = 'You are helpful assistant. When user gives any input, you should generate 5 synonyms in comma seperated format'
human_template = "{input}"
chatprompt = ChatPromptTemplate.from_messages([
("system", template),
('human',human_template)
])

chain = chatprompt | chat_llm | commaseperateop()
chain.invoke({"input":"bad"})

['awful', ' terrible', ' poor', ' inferior', ' subpar']